# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 2:24PM,243192,10,EYE42979,Eye Pharma Inc,Released
1,Aug 1 2022 2:24PM,243192,10,EYE42980,Eye Pharma Inc,Released
2,Aug 1 2022 2:24PM,243192,10,EYE42981,Eye Pharma Inc,Released
3,Aug 1 2022 2:21PM,243191,10,8347814,"Herbivore Botanicals, LLC.",Released
4,Aug 1 2022 2:16PM,243188,16,AIC-8347810,American International Chemical,Released
5,Aug 1 2022 2:15PM,243190,10,8347811,"ClearSpec, LLC",Released
6,Aug 1 2022 2:15PM,243189,10,0085919737,ISDIN Corporation,Released
7,Aug 1 2022 2:15PM,243189,10,0085919738,ISDIN Corporation,Released
8,Aug 1 2022 2:15PM,243189,10,0085919756,ISDIN Corporation,Released
9,Aug 1 2022 2:15PM,243189,10,0085919617,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,243188,Released,1
48,243189,Released,4
49,243190,Released,1
50,243191,Released,1
51,243192,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243188,NaN,NaN,1.0
243189,NaN,NaN,4.0
243190,NaN,NaN,1.0
243191,NaN,NaN,1.0
243192,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0.0,0.0,1.0
243101,0.0,1.0,0.0
243110,0.0,0.0,3.0
243123,0.0,1.0,1.0
243129,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0,0,1
243101,0,1,0
243110,0,0,3
243123,0,1,1
243129,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,0,0,1
1,243101,0,1,0
2,243110,0,0,3
3,243123,0,1,1
4,243129,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,,,1
1,243101,,1,
2,243110,,,3
3,243123,,1,1
4,243129,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc
3,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC."
4,Aug 1 2022 2:16PM,243188,16,American International Chemical
5,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC"
6,Aug 1 2022 2:15PM,243189,10,ISDIN Corporation
10,Aug 1 2022 2:12PM,243187,10,"Methapharm, Inc."
14,Aug 1 2022 2:12PM,243186,10,"Methapharm, Inc."
15,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc."
16,Aug 1 2022 2:08PM,243184,10,ISDIN Corporation
29,Aug 1 2022 2:04PM,243177,10,"Akron BioProducts, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,,,3
1,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC.",,,1
2,Aug 1 2022 2:16PM,243188,16,American International Chemical,,,1
3,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC",,,1
4,Aug 1 2022 2:15PM,243189,10,ISDIN Corporation,,,4
5,Aug 1 2022 2:12PM,243187,10,"Methapharm, Inc.",,,4
6,Aug 1 2022 2:12PM,243186,10,"Methapharm, Inc.",,,1
7,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc.",,,1
8,Aug 1 2022 2:08PM,243184,10,ISDIN Corporation,,,13
9,Aug 1 2022 2:04PM,243177,10,"Akron BioProducts, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3,,
1,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC.",1,,
2,Aug 1 2022 2:16PM,243188,16,American International Chemical,1,,
3,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC",1,,
4,Aug 1 2022 2:15PM,243189,10,ISDIN Corporation,4,,
5,Aug 1 2022 2:12PM,243187,10,"Methapharm, Inc.",4,,
6,Aug 1 2022 2:12PM,243186,10,"Methapharm, Inc.",1,,
7,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc.",1,,
8,Aug 1 2022 2:08PM,243184,10,ISDIN Corporation,13,,
9,Aug 1 2022 2:04PM,243177,10,"Akron BioProducts, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3,,
1,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC.",1,,
2,Aug 1 2022 2:16PM,243188,16,American International Chemical,1,,
3,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC",1,,
4,Aug 1 2022 2:15PM,243189,10,ISDIN Corporation,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3.0,NaN,NaN
1,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC.",1.0,NaN,NaN
2,Aug 1 2022 2:16PM,243188,16,American International Chemical,1.0,NaN,NaN
3,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC",1.0,NaN,NaN
4,Aug 1 2022 2:15PM,243189,10,ISDIN Corporation,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3.0,0.0,0.0
1,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC.",1.0,0.0,0.0
2,Aug 1 2022 2:16PM,243188,16,American International Chemical,1.0,0.0,0.0
3,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC",1.0,0.0,0.0
4,Aug 1 2022 2:15PM,243189,10,ISDIN Corporation,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5106540,150.0,26.0,19.0
15,972627,13.0,37.0,0.0
16,972621,8.0,3.0,0.0
19,1702044,7.0,24.0,0.0
20,1215648,81.0,19.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5106540,150.0,26.0,19.0
1,15,972627,13.0,37.0,0.0
2,16,972621,8.0,3.0,0.0
3,19,1702044,7.0,24.0,0.0
4,20,1215648,81.0,19.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,150.0,26.0,19.0
1,15,13.0,37.0,0.0
2,16,8.0,3.0,0.0
3,19,7.0,24.0,0.0
4,20,81.0,19.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,150.0
1,15,Released,13.0
2,16,Released,8.0
3,19,Released,7.0
4,20,Released,81.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,19.0,0.0,0.0,0.0,0.0
Executing,26.0,37.0,3.0,24.0,19.0
Released,150.0,13.0,8.0,7.0,81.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,19.0,0.0,0.0,0.0,0.0
1,Executing,26.0,37.0,3.0,24.0,19.0
2,Released,150.0,13.0,8.0,7.0,81.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,19.0,0.0,0.0,0.0,0.0
1,Executing,26.0,37.0,3.0,24.0,19.0
2,Released,150.0,13.0,8.0,7.0,81.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()